In [79]:
from datetime import timedelta, date, datetime
import time
import pandas as pd
import requests
import numpy as np
def limit(limit,timeframe):
    
    timeframes = (['1m',1,'minutes',limit*1],['3m',3,'minutes',limit*3],
                  ['5m',5,'minutes',limit*5],['15m',15,'minutes',limit*15],
                  ['30m',30,'minutes',limit*30],['1h',1,'hours',limit*1],
                  ['2h',2,'hours',limit*2],['4h',4,'hours',limit*4],
                  ['6h',6,'hours',limit*6],['8h',8,'hours',limit*8],
                  ['12h',12,'hours',limit*12],['1d',1,'days',limit*1],
                  ['3d',3,'days',limit*3],['1w',1,'weeks',limit*7])
    timeframes = pd.DataFrame(timeframes,columns=['timeframe' , 'time' , 'name_time' , 'limit'])
    timeframes.set_index('timeframe', inplace=True)

    timeframes['limit']  = timeframes['limit'].astype(np.float64)
    
    now = int(round(time.time() * 1000.0))
    if timeframes.loc[timeframe, 'name_time'] == 'minutes':
        from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(minutes=timeframes.loc[timeframe, 'limit'])

    if timeframes.loc[timeframe, 'name_time'] == 'hours':
        from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(hours=timeframes.loc[timeframe, 'limit'])

    if timeframes.loc[timeframe, 'name_time'] == 'days':
        from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(days=timeframes.loc[timeframe, 'limit'])



    from_datetime=datetime.strftime(from_datetime, '%Y-%m-%d %H:%M:%S')
    todate = from_datetime
    return todate
todate=limit(400,"1d")
todate

'2021-02-13 22:22:23'

# margin

In [2]:
import time
from datetime import timedelta, date, datetime
import time
import pandas as pd
import requests
import numpy as np
def number_candle(limit,timeframe):

        timeframes = (['1m',1,'minutes',limit*1],['3m',3,'minutes',limit*3],
                      ['5m',5,'minutes',limit*5],['15m',15,'minutes',limit*15],
                      ['30m',30,'minutes',limit*30],['1h',1,'hours',limit*1],
                      ['2h',2,'hours',limit*2],['4h',4,'hours',limit*4],
                      ['6h',6,'hours',limit*6],['8h',8,'hours',limit*8],
                      ['12h',12,'hours',limit*12],['1d',1,'days',limit*1],
                      ['3d',3,'days',limit*3],['1w',1,'weeks',limit*7])
        timeframes = pd.DataFrame(timeframes,columns=['timeframe' , 'time' , 'name_time' , 'limit'])
        timeframes.set_index('timeframe', inplace=True)

        timeframes['limit']  = timeframes['limit'].astype(np.float64)

        now = int(round(time.time() * 1000.0))
        if timeframes.loc[timeframe, 'name_time'] == 'minutes':
            from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(minutes=timeframes.loc[timeframe, 'limit'])

        if timeframes.loc[timeframe, 'name_time'] == 'hours':
            from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(hours=timeframes.loc[timeframe, 'limit'])

        if timeframes.loc[timeframe, 'name_time'] == 'days':
            from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(days=timeframes.loc[timeframe, 'limit'])



        from_datetime=datetime.strftime(from_datetime, '%Y-%m-%d %H:%M:%S')
        fromdate = from_datetime
        return fromdate

def ohlcvs_margin_kucoin(symbol,timeframe,limit):







        symbol     =  symbol
        tf         =  {'1m':'1min','3m':'3min','5m':'5min',
                       '15m':'15min','30m':'30min','1h':'1hour','2h':'2hour','4h':'4hour','6h':'6hour'
                      ,'8h':'8hour','12h':'12hour','1d':'1day','1w':'1week'}
        duration   =  tf[timeframe]
        fromdate   =  datetime.strptime(number_candle(limit,timeframe),'%Y-%m-%d %H:%M:%S').strftime("%d/%m/%Y %H:%M:%S")
        fromdate   =  int(time.mktime(datetime.strptime(str(fromdate), "%d/%m/%Y %H:%M:%S").timetuple()))

        todate     =  int((time.mktime((datetime.now()).timetuple())))

        url = "https://api.kucoin.com/api/v1/market/candles?type=%s&symbol=%s&startAt=%s&endAt=%s" % (duration,symbol,fromdate,todate)

        payload={}
        files={}
        headers ={}

        df = requests.request("GET", url, headers=headers, data=payload, files=files)
        df=df.json()
        df=df['data']

        df=pd.DataFrame(df)
        df = df.rename({0:"Timestamp",1:"open",
                        2:"close",3:"high",4:"low",5:"volume",6:"value"}, axis='columns')
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
        df['date'] = df['Timestamp'].dt.strftime("%d/%m/%Y")
        df['time-utc'] = df['Timestamp'].dt.strftime("%H:%M:%S")
        df.drop(['value','Timestamp'], inplace=True, axis=1)
        df=df[["date", "time-utc", "open", "high", "low", "close", "volume"]] 
        df =df[::-1]#reverse dataframe
        df.reset_index(drop=True,inplace=True)
        return df


symbol    =  'BTC-USDT'
timeframe =  '30m'
limit     =  400

df = ohlcvs_margin_kucoin(symbol,timeframe,limit)
df

,date,time-utc,open,high,low,close,volume
0,16/03/2022,04:00:00,39158.2,39208.9,38934,39082.1,137.45943926
1,16/03/2022,04:30:00,39082,39191.4,39000.7,39116.8,50.79739142
2,16/03/2022,05:00:00,39116.8,39128.3,38982,39050.8,71.60449328
3,16/03/2022,05:30:00,39053.8,39418.7,39005,39321.6,102.71233492
4,16/03/2022,06:00:00,39320.9,39579.9,39263.2,39345.3,138.88186555
...,...,...,...,...,...,...,...
393,24/03/2022,08:30:00,43035.8,43169,43034,43045.1,82.21603006
394,24/03/2022,09:00:00,43045.1,43169,43040,43110.7,82.29394678
395,24/03/2022,09:30:00,43110.7,43214,42843,43023.4,251.89917531
396,24/03/2022,10:00:00,43023.5,43048.8,42925.6,43000.4,90.85180572


# future

In [8]:
int(round(time.time() * 1000.0))
dt_obj = datetime.strptime('19.03.2022 09:38:42,76',
                           '%d.%m.%Y %H:%M:%S,%f')
millisec = dt_obj.timestamp() * 1000
millisec
1

s = 1647853200000 / 1000.0
datetime.fromtimestamp(s).strftime('%Y-%m-%d %H:%M:%S')
frome=datetime.fromtimestamp(1647975722272/1000.0)
to=datetime.fromtimestamp(1647259322000/1000.0)
print(frome,'\n',to,'\n',datetime.fromtimestamp(1646539322000/1000.0))

2022-03-22 23:32:02.272000 
 2022-03-14 15:32:02 
 2022-03-06 07:32:02


In [86]:
import pandas as pd
import requests
url = "https://api.kucoin.com/api/v1/timestamp"

payload={}
files={}
headers ={}

df = requests.request("GET", url, headers=headers, data=payload, files=files)
df=df.json()
df

{'code': '200000', 'data': 1647858102887}

In [11]:
fruits = ['apple', 'banana', 'cherry']
names = ["Jimmy", "Timmy",["Kenny", "Lenny"]]
fruits.extend(names)
fruits
pd.set_option('display.max_rows', df.shape[0]+1)
df


,date,time,open,high,low,close,volume
0,06/03/2022,05:00:00,39474.0,39670.0,39427.0,39497.0,278115
1,06/03/2022,06:00:00,39488.0,39610.0,39370.0,39480.0,265268
2,06/03/2022,07:00:00,39488.0,39515.0,39269.0,39395.0,291656
3,06/03/2022,08:00:00,39348.0,39370.0,39113.0,39144.0,148421
4,06/03/2022,09:00:00,39171.0,39171.0,38642.0,38688.0,593831
5,06/03/2022,10:00:00,38690.0,38722.0,38117.0,38270.0,4184218
6,06/03/2022,11:00:00,38250.0,38374.0,38102.0,38360.0,2015908
7,06/03/2022,12:00:00,38369.0,38682.0,38311.0,38514.0,2763912
8,06/03/2022,13:00:00,38506.0,39046.0,38506.0,38904.0,1305020
9,06/03/2022,14:00:00,38900.0,38964.0,38742.0,38825.0,4631924


In [10]:
# لیمیت هر چقدر بود باید به بازه 200 تا بشکنیم و داده ها را بگیریم
#باید لیمیت را بشکنیم و زمانهای مربوط به 200 کندل را خارج کنیم و در یک لیست قرار بدیم و  از آنها استفاده کنیم
import time
from datetime import timedelta, date, datetime
import time
import pandas as pd
import requests
import numpy as np
def kucoin_future(symbol,limit,timeframe):
    pairs = pd.read_pickle("pairs_kucoin_future.pkl")
    save_pairs=pairs.copy()

    def number_candle(limit,timeframe):
        i=0
        list_date=[]
        n=limit/200
        while i<n:
                timeframes = (['1m',1,'minutes',limit*1],['3m',3,'minutes',limit*3],
                              ['5m',5,'minutes',limit*5],['15m',15,'minutes',limit*15],
                              ['30m',30,'minutes',limit*30],['1h',1,'hours',limit*1],
                              ['2h',2,'hours',limit*2],['4h',4,'hours',limit*4],
                              ['6h',6,'hours',limit*6],['8h',8,'hours',limit*8],
                              ['12h',12,'hours',limit*12],['1d',1,'days',limit*1],
                              ['3d',3,'days',limit*3],['1w',1,'weeks',limit*7])
                timeframes = pd.DataFrame(timeframes,columns=['timeframe' , 'time' , 'name_time' , 'limit'])
                timeframes.set_index('timeframe', inplace=True)

                timeframes['limit']  = timeframes['limit'].astype(np.float64)

                now = int(round(time.time() * 1000.0))
                if timeframes.loc[timeframe, 'name_time'] == 'minutes':
                    from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(minutes=timeframes.loc[timeframe, 'limit'])

                if timeframes.loc[timeframe, 'name_time'] == 'hours':
                    from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(hours=timeframes.loc[timeframe, 'limit'])

                if timeframes.loc[timeframe, 'name_time'] == 'days':
                    from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(days=timeframes.loc[timeframe, 'limit'])



                from_datetime=datetime.strftime(from_datetime, '%Y-%m-%d %H:%M:%S')
                from_datetime=datetime.strptime(from_datetime,'%Y-%m-%d %H:%M:%S')
                fromdate = from_datetime
                fromdate =  int(from_datetime.timestamp() * 1000)
                list_date.append(fromdate)
                limit=limit-200
                if limit<200:
                    todate =  int(round(time.time() * 1000.0))
                    list_date.append(todate)
#                     list_date.reverse()
                    break
                i+=1
        return list_date

    i=0
    df=[]
    list_date=number_candle(limit,timeframe)
    while i<(len(list_date)-1):

            symbol  =  symbol
            tf =  {'1m':1,'5m':5,
                           '15m':15,'30m':30,'1h':60,'2h':120,'4h':240
                          ,'8h':480,'12h':720,'1d':1440,'1w':10080}
            duration =  tf[timeframe]
            fromdate = list_date[i]

            todate = list_date[i+1]

            url = "https://api-futures.kucoin.com/api/v1/kline/query?symbol=%s&granularity=%s&from=%s&to=%s"\
            % (symbol,duration,fromdate,todate)

            payload={}
            files={}
            headers ={}

            dff = requests.request("GET", url, headers=headers, data=payload, files=files)
            dff=dff.json()
            dff=dff['data']
            df.extend(dff)
            i+=1
    df=pd.DataFrame(df)
    df = df.rename({0:"Timestamp",1:"open",
                    2:"high",3:"low",4:"close",5:"volume"}, axis='columns')
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit='ms')
    df['date'] = df["Timestamp"].dt.strftime("%d/%m/%Y")
    df['time'] = df["Timestamp"].dt.strftime("%H:%M:%S")
    df.drop(['Timestamp'], inplace=True, axis=1)
    df=df[["date", "time", "open", "high", "low", "close", "volume"]] 


    return df


symbol='XBTUSDM'
limit=400
timeframe='1h'
df=kucoin_future(symbol,limit,timeframe)
df

,date,time,open,high,low,close,volume
0,06/03/2022,05:00:00,39474.0,39670.0,39427.0,39497.0,278115
1,06/03/2022,06:00:00,39488.0,39610.0,39370.0,39480.0,265268
2,06/03/2022,07:00:00,39488.0,39515.0,39269.0,39395.0,291656
3,06/03/2022,08:00:00,39348.0,39370.0,39113.0,39144.0,148421
4,06/03/2022,09:00:00,39171.0,39171.0,38642.0,38688.0,593831
...,...,...,...,...,...,...,...
394,22/03/2022,15:00:00,42774.0,42799.0,42562.0,42575.0,502643
395,22/03/2022,16:00:00,42570.0,42698.0,42475.0,42514.0,455041
396,22/03/2022,17:00:00,42512.0,42649.0,42477.0,42601.0,1270578
397,22/03/2022,18:00:00,42618.0,42636.0,42506.0,42603.0,349749


In [107]:
a = [1, 2, 3]
b = [4, 5, 6]
c = [7, 8, 9]
a.extend(c)
a.extend(b)
a

[1, 2, 3, 7, 8, 9, 4, 5, 6]

In [10]:
import pandas as pd
i=33
url = "https://api-futures.kucoin.com/api/v1/kline/query?symbol=%s&granularity=%s&from=%s&to=%s"\
% ('.KXBTUSDT',60,t[i],t[i+1])

payload={}
files={}
headers ={}

dff = requests.request("GET", url, headers=headers, data=payload, files=files)
df=dff.json()
df=df['data']
df=pd.DataFrame(df)
df = df.rename({0:"Timestamp",1:"open",
        2:"high",3:"low",4:"close",5:"volume"}, axis='columns')
df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit='ms')
df['date'] = df["Timestamp"].dt.strftime("%d/%m/%Y")
df['time'] = df["Timestamp"].dt.strftime("%H:%M:%S")
df.drop(['Timestamp'], inplace=True, axis=1)
df=df[["date", "time", "open", "high", "low", "close", "volume"]]
df

,date,time,open,high,low,close,volume
0,27/02/2022,15:00:00,38788.86,39427.09,38744.19,39356.58,0
1,27/02/2022,16:00:00,39358.67,39443.14,38950.09,39090.20,0
2,27/02/2022,17:00:00,39090.20,39126.83,38948.89,38971.93,0
3,27/02/2022,18:00:00,38971.93,39146.40,38811.63,38916.51,0
4,27/02/2022,19:00:00,38916.64,38965.08,37522.02,37605.80,0
5,27/02/2022,20:00:00,37604.58,38285.96,37408.49,37953.96,0
6,27/02/2022,21:00:00,37957.68,38035.73,37012.16,37418.92,0
7,27/02/2022,22:00:00,37418.10,37887.92,37354.96,37808.32,0
8,27/02/2022,23:00:00,37813.48,37816.43,37364.76,37691.32,0
9,28/02/2022,00:00:00,37695.34,37801.83,37450.53,37558.36,0


In [4]:
import time
from datetime import timedelta, date, datetime
import time
import pandas as pd
import requests
import numpy as np
def number_candle(limit,timeframe):
            i=0
            list_date=[]
            n=limit/200
            while i<n:
                    timeframes = (['1m',1,'minutes',limit*1],['3m',3,'minutes',limit*3],
                                  ['5m',5,'minutes',limit*5],['15m',15,'minutes',limit*15],
                                  ['30m',30,'minutes',limit*30],['1h',1,'hours',limit*1],
                                  ['2h',2,'hours',limit*2],['4h',4,'hours',limit*4],
                                  ['6h',6,'hours',limit*6],['8h',8,'hours',limit*8],
                                  ['12h',12,'hours',limit*12],['1d',1,'days',limit*1],
                                  ['3d',3,'days',limit*3],['1w',1,'weeks',limit*7])
                    timeframes = pd.DataFrame(timeframes,columns=['timeframe' , 'time' , 'name_time' , 'limit'])
                    timeframes.set_index('timeframe', inplace=True)

                    timeframes['limit']  = timeframes['limit'].astype(np.float64)

                    now = int(round(time.time() * 1000.0))
                    if timeframes.loc[timeframe, 'name_time'] == 'minutes':
                        from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(minutes=timeframes.loc[timeframe, 'limit'])

                    if timeframes.loc[timeframe, 'name_time'] == 'hours':
                        from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(hours=timeframes.loc[timeframe, 'limit'])

                    if timeframes.loc[timeframe, 'name_time'] == 'days':
                        from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(days=timeframes.loc[timeframe, 'limit'])



                    from_datetime=datetime.strftime(from_datetime, '%Y-%m-%d %H:%M:%S')
                    from_datetime=datetime.strptime(from_datetime,'%Y-%m-%d %H:%M:%S')
                    fromdate = from_datetime
                    fromdate =  int(from_datetime.timestamp() * 1000)
                    list_date.append(fromdate)
                    limit=limit-200
                    if limit<200:
                        todate =  int(round(time.time() * 1000.0))
                        list_date.append(todate)
                        break
                    i+=1
            return list_date
t=number_candle(9624,'1h')

In [35]:
from datetime import timedelta, date, datetime
import time
from_datetime='2022-01-01'

#from_datetime=time.strftime(from_datetime, '%Y-%m-%d')
from_timestamp = exchange.parse8601(from_datetime)
ohlcvs = exchange.fetch_ohlcv('BCH/USDT:USDT', from_timestamp)

KeyError: None